In [ ]:
!pip install wandb

In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D
from tensorflow.keras import Model
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import wandb
from wandb.keras import WandbCallback
import numpy as np

In [21]:
wandb.login()

True

In [23]:
wandb.init(project="Face-Detection", entity="gharabadiyan")

In [24]:
config = wandb.config
config.learning_rate = 0.001

In [ ]:
train_data_path = '/content/drive/MyDrive/7-7 dataset'

width = height = 224
batch_size = 32
epochs=15

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    brightness_range = (0.6,1.2),
    zoom_range = 0.1,
    shear_range = 0.3,
    rotation_range = 15,
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    train_data_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"
)
val_data = idg.flow_from_directory(
    train_data_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "validation"
)



Found 1090 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


In [ ]:
class FaceNet(Model):
  def __init__(self):
    super().__init__()
    
    self.dense_1 = Dense(128,activation='relu')
    self.dense_2 = Dense(14,activation='softmax')
    self.dense_3 = Dense(32,activation='relu')
    self.conv2d_1 = Conv2D(64,(3,3),activation='relu',input_shape=(width, height, 3))
    self.flatten = Flatten()
    self.conv2d_2 = Conv2D(64,(5,5),activation='relu')
    self.max_pool = MaxPool2D()

  def call(self,x):
    k = self.conv2d_1(x)
    y = self.max_pool(k)
    z = self.conv2d_2(y)
    u = self.flatten(y)
    c = self.dense_1(u)
    a = self.dense_3(c)
    out = self.dense_2(a)

    return out


In [ ]:
model = FaceNet()

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

loss_function = tf.keras.losses.CategoricalCrossentropy()

In [26]:
train_accuracy = tf.keras.metrics.CategoricalAccuracy()
test_accuracy = tf.keras.metrics.CategoricalAccuracy()

train_loss = tf.keras.metrics.MeanAbsoluteError()
test_loss = tf.keras.metrics.MeanAbsoluteError()

In [29]:
# train loop
# fit function

for epoch in range(epochs):
  train_accuracy.reset_states()
  test_accuracy.reset_states()
  train_loss.reset_states()
  test_loss.reset_states()

  print('epoch:',epoch)
  #Training ---------------------------------------------
  for i ,(images, labels) in enumerate(tqdm(train_data)):
    if len(train_data) <= i:
            break
    with tf.GradientTape() as gTape:
      predictions = model(images)
      loss = loss_function(labels,predictions)
      train_loss(labels,predictions)
      train_accuracy(labels,predictions)

    # محاسبات مشتقات 
    gradients = gTape.gradient(loss,model.trainable_variables,unconnected_gradients=tf.UnconnectedGradients.ZERO)


    # به روز رسانی وزن ها 
    optimizer.apply_gradients([
    (grad, var) 
    for (grad, var) in zip(gradients, model.trainable_variables) 
    if grad is not None
])
  #Validation -----------------------------------------------
  for i ,(images, labels) in enumerate(tqdm(val_data)):
    if len(val_data) <= i:
            break
    predictions = model(images)
    loss = loss_function(labels,predictions)
    test_loss(labels,predictions)
    test_accuracy(labels,predictions)



  print('Train Accuracy:',train_accuracy.result())
  print('Test Accuracy:',test_accuracy.result())
  print('Train loss:',train_loss.result())
  print('Test loss:',test_loss.result())
  wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss.result()),
                   'acc': float(train_accuracy.result()), 
                   'val_loss': np.mean(test_loss.result()),
                   'val_acc':float(test_accuracy.result())})


epoch: 0


100%|██████████| 9/9 [00:07<00:00,  1.13it/s]


Train Accuracy: tf.Tensor(0.89816517, shape=(), dtype=float32)
Test Accuracy: tf.Tensor(0.73880595, shape=(), dtype=float32)
Train loss: tf.Tensor(0.0233667, shape=(), dtype=float32)
Test loss: tf.Tensor(0.04238562, shape=(), dtype=float32)
epoch: 1


100%|██████████| 9/9 [00:07<00:00,  1.23it/s]


Train Accuracy: tf.Tensor(0.9321101, shape=(), dtype=float32)
Test Accuracy: tf.Tensor(0.79850745, shape=(), dtype=float32)
Train loss: tf.Tensor(0.016812913, shape=(), dtype=float32)
Test loss: tf.Tensor(0.035126396, shape=(), dtype=float32)
epoch: 2


...

Train Accuracy: tf.Tensor(0.9678899, shape=(), dtype=float32)
Test Accuracy: tf.Tensor(0.78731346, shape=(), dtype=float32)
Train loss: tf.Tensor(0.0077681034, shape=(), dtype=float32)
Test loss: tf.Tensor(0.034146745, shape=(), dtype=float32)
epoch: 13


100%|██████████| 9/9 [00:07<00:00,  1.15it/s]


Train Accuracy: tf.Tensor(0.97431195, shape=(), dtype=float32)
Test Accuracy: tf.Tensor(0.8208955, shape=(), dtype=float32)
Train loss: tf.Tensor(0.0070416224, shape=(), dtype=float32)
Test loss: tf.Tensor(0.027857073, shape=(), dtype=float32)
epoch: 14


100%|██████████| 9/9 [00:07<00:00,  1.14it/s]

Train Accuracy: tf.Tensor(0.9477064, shape=(), dtype=float32)
Test Accuracy: tf.Tensor(0.76119405, shape=(), dtype=float32)
Train loss: tf.Tensor(0.011436544, shape=(), dtype=float32)
Test loss: tf.Tensor(0.03628528, shape=(), dtype=float32)


In [32]:
model.save_weights('detect_face.h5')